# Video: Finding and Imputing Missing Data

This video walks through basic checks for missing data, and common ways to fill it in.

In [ ]:
import pandas as pd


In [ ]:
penguins_adelie = pd.read_csv("https://portal.edirepository.org/nis/dataviewer?packageid=knb-lter-pal.219.3&entityid=002f3893385f710df69eeebe893144ff", index_col="Sample Number")
penguins_gentoo = pd.read_csv("https://portal.edirepository.org/nis/dataviewer?packageid=knb-lter-pal.220.3&entityid=e03b43c924f226486f2f0ab6709d2381", index_col="Sample Number")
penguins_chinstrap = pd.read_csv("https://portal.edirepository.org/nis/dataviewer?packageid=knb-lter-pal.221.2&entityid=fe853aa8f7a59aa84cdd3197619ef462", index_col="Sample Number")
penguins = pd.concat([penguins_adelie, penguins_gentoo, penguins_chinstrap])
penguins.head()

In [ ]:
penguins.hist()

In [ ]:
penguins.count()

In [ ]:
penguins.isna().sum()

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imputer = SimpleImputer(strategy="mean")

In [ ]:
imputer.fit(penguins)

In [ ]:
penguins.info()

In [ ]:
penguins_numeric_columns = [c for c in penguins.columns if penguins[c].dtype == "float64"]
penguins_numeric_columns

In [ ]:
imputer = SimpleImputer(strategy="mean")
imputer.fit(penguins[penguins_numeric_columns])

In [ ]:
imputer.transform(penguins[penguins_numeric_columns])

In [ ]:
penguins_new = penguins.copy()
penguins_new[penguins_numeric_columns] = imputer.transform(penguins[penguins_numeric_columns])

In [ ]:
penguins_new.head()

In [ ]:
penguins.head()